In [9]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import seaborn as sns
from scipy import stats
from utils.util_functions import *

In [20]:
# Data agencia nacional de seguridad vial (ANSV)
df_ansv = pd.read_csv("../../../colombia_speeds/new_data/Datos_Velocidad_Colombia/tunja_briceno_coordinates_speed_info.csv", sep=";")
df_ansv = df_ansv[df_ansv["SEGMENTO"].notna()]
del df_ansv["Unnamed: 13"]
df_ansv["SEGMENTO"] = df_ansv["SEGMENTO"].astype(int)
df_ansv

,indice,Ruta,estacion,media,desviacion,moda,mediana,velmax,velmin,percentil98,percentil85,percentil15,SEGMENTO
0,1,6210,16+370,64.0594059,10.00082175,65,65,92,28,80,73,56,198
1,2,6210,5+410,58.8181818,12.66293805,51,60,93,18,81,71,44,171
6,7,5502,45+550,67.8918919,15.91522402,80,71,104,23,94,80.95,51.05,194
7,8,5502,38+050,65.8322148,11.31394899,58,66,97,36,88.04,76,53,168
8,9,5502,34+700,73.556338,13.60356891,80,74,109,6,100,84,61,193
11,12,5501,104+990,69.4854369,19.52116201,73,69,131,29,106.92,88,49.9,184
12,13,5501,94+700,65.3137255,14.5288926,60,64,112,27,92.96,78,51,182
13,1,6210,16+370,60.034,12.573,61,61,93,22,91.12,70.8,49,149
14,2,6210,5+410,59.124,8.994,58,59,98,38,77.36,66.4,50,122
19,7,5502,45+550,70.027,16.247,59,68,117,6,99.04,86.8,56.2,145


In [22]:
df_ansv

,indice,Ruta,estacion,media,desviacion,moda,mediana,velmax,velmin,percentil98,percentil85,percentil15,SEGMENTO
0,1,6210,16+370,64.0594059,10.00082175,65,65,92,28,80,73,56,198
1,2,6210,5+410,58.8181818,12.66293805,51,60,93,18,81,71,44,171
6,7,5502,45+550,67.8918919,15.91522402,80,71,104,23,94,80.95,51.05,194
7,8,5502,38+050,65.8322148,11.31394899,58,66,97,36,88.04,76,53,168
8,9,5502,34+700,73.556338,13.60356891,80,74,109,6,100,84,61,193
11,12,5501,104+990,69.4854369,19.52116201,73,69,131,29,106.92,88,49.9,184
12,13,5501,94+700,65.3137255,14.5288926,60,64,112,27,92.96,78,51,182
13,1,6210,16+370,60.034,12.573,61,61,93,22,91.12,70.8,49,149
14,2,6210,5+410,59.124,8.994,58,59,98,38,77.36,66.4,50,122
19,7,5502,45+550,70.027,16.247,59,68,117,6,99.04,86.8,56.2,145


In [26]:
df_dis = get_all_dis()
df_dis.loc[:,'date_1']=df_dis['timestamp'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S')) #convert str to timestamp
df_dis['date_3h'] = df_dis['date_1'].apply(lambda dt: datetime(dt.year, dt.month, dt.day, 3*(dt.hour // 3)))
df_dis["hourTimeStamp"] = df_dis["date_3h"].dt.hour

df_dis = df_dis.groupby(["id","hourTimeStamp"]).agg(
    speed = ("speed",stats.hmean),
    speed_std = ("speed","std")).reset_index()

# df_dis = df_dis[(df_dis["speed"].notna()) & (df_dis["speed"] < 150) & (df_dis["speed"] >15)] #Pues hay velocidades muy altas, ej: id 135 el 07-05 de 2022
    
# MERGE INDICES DISAGGREGATED-V2V3
df_v2v3 = pd.read_csv("../../idv2_idv3.csv") # id_2 Road_2 son las columnas de los segmentos que estamos analizando
df_dis = df_dis.merge(df_v2v3, left_on = "id", right_on = "id")


df_dis = df_dis.groupby(["id"]).agg(
    speed = ("speed","mean"),
    speed_median = ('speed', "median"),
    speed_std = ("speed","std"),
    speed_max = ('speed', "max"),
    speed_min = ('speed', "min")).reset_index()

df_dis = df_dis.merge(df_ansv, left_on = "id", right_on = "SEGMENTO")

# df_dis["hourTimeStamp"] = df_dis["hourTimeStamp"].replace([3,6,9,12,15,18,21,24],["3-6","6-9","9-12","12-15","15-18","18-21","21-24","0-3"])
df_dis

,id,speed,speed_median,speed_std,speed_max,speed_min,indice,Ruta,estacion,media,desviacion,moda,mediana,velmax,velmin,percentil98,percentil85,percentil15,SEGMENTO
0,119,90.180106,90.180106,1.640928e-14,90.180106,90.180106,8,5502,38+050,70.817,18.248,66,71,137,4,103.24,85,58,119
1,122,29.116217,29.572565,2.931453e+00,31.916394,25.449070,2,6210,5+410,59.124,8.994,58,59,98,38,77.36,66.4,50,122
2,133,62.019001,56.827401,1.982967e+01,86.349742,34.527426,13,5501,94+700,51.827,13.885,56,53,83,19,79,66.1,36.35,133
3,135,82.707594,85.354652,1.670083e+01,102.502653,61.781565,12,5501,104+990,80.176,16.208,88,83,126,17,106.6,95,64.5,135


In [8]:
geodf_buffer = gpd.GeoDataFrame.from_file("../../../shared_speeds_wbg/buffer_without_overlaps/buffer_without_overlaps.shp")

In [12]:
df_geo = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitud, df.Latitud)) # X,Y coordinates in longitude and latitude

df_in_buffer = gpd.sjoin(df_geo, geodf_buffer)    

df_in_buffer = df_in_buffer[df_in_buffer["ID_1"].notna()]
df_in_buffer

/var/folders/vk/14ym66kj4_d9xkr_w6f6cr7w0000gn/T/ipykernel_66907/962902177.py:3: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  df_in_buffer = gpd.sjoin(df_geo, geodf_buffer)


,ID,Ruta,Estación (PR),Latitud,Longitud,Descripción,Tramo Homogéneo,Cantidad de Lecturas,geometry,index_right,id,Country,Road,Length,START_X,START_Y,END_X,END_Y,ID_1,Area
0,1,6210,16+370,5.76290,-72.91407,Entrada Nobsa,10,101,POINT (-72.91407 5.76290),831,0,None,None,0.000,0.000000,0.000000,0.000000,0.000000,198|149,26279.253896
1,2,6210,5+410,5.79070,-72.98645,Salida Nobsa,10,121,POINT (-72.98645 5.79070),663,171,Colombia,Briceno_Tunja_Sogamoso,4397.598,-72.972519,5.768947,-72.997976,5.796194,171,34020.921194
6,7,5502,45+550,5.79906,-73.04251,Salida Duitama,8,148,POINT (-73.04251 5.79906),804,194,Colombia,Briceno_Tunja_Sogamoso,4371.211,-73.027516,5.807177,-73.058737,5.789515,194,59401.388348
7,8,5502,38+050,5.77863,-73.10527,Entrada Paipa,8,149,POINT (-73.10527 5.77863),646,168,Colombia,Briceno_Tunja_Sogamoso,3142.038,-73.082445,5.787136,-73.108278,5.778494,168,45407.112102
8,9,5502,34+700,5.77761,-73.13197,Salida Paipa,7,142,POINT (-73.13197 5.77761),799,193,Colombia,Briceno_Tunja_Sogamoso,3641.319,-73.127496,5.778922,-73.156826,5.764595,193,50805.190340
11,12,5501,104+990,5.42993,-73.45025,Tierranegra,2,103,POINT (-73.45025 5.42993),740,0,None,None,0.000,0.000000,0.000000,0.000000,0.000000,184|135,13367.039669
12,13,5501,92+700,5.36627,-73.53035,Ventaquemada,1,91,POINT (-73.53035 5.36627),730,182,Colombia,Briceno_Tunja_Sogamoso,3782.569,-73.522355,5.372443,-73.546987,5.351387,182,55466.216144


# Waze

In [21]:
df_waze = pd.read_csv("../../../colombia_speeds/new_data/ANSV_Colombia_Data/vel_data_waze.csv")
df_waze

,OBJECTID,nameroute,idroute,toname,historictime,fromname,length0,jamlevel,time,subroutes_childindex,...,fecha_hora,fecha,mes,dia,hora,minuto,dia_semana,lenght_mt,vel_historic,Shape_Length
0,1,Bogota_Tunja,28327,Tunja,5021,A Sopó,108426.0,0.0,5039.0,0.0,...,2021-11-30 19:16:18,2021-11-30,Nov,30,19:16:18,16,Tue,108.426,77.740211,0.980950
1,2,Bogota_Tunja,28327,Tunja-Paipa / RN55-02 >(N),226,Tunja,5629.0,1.0,226.0,1.0,...,2021-11-30 19:16:18,2021-11-30,Nov,30,19:16:18,16,Tue,5.629,89.665487,0.051356
2,3,Duitama_Sogamoso,28847,Calle 11,880,Duitama-Tibasosa / RN55,15271.0,0.0,880.0,2.0,...,2021-11-30 20:01:26,2021-11-30,Nov,30,20:01:26,1,Tue,15.271,62.472273,0.138197
3,4,Duitama_Sogamoso,28847,Duitama-Paipa / RN55-02 >(S),80,Duitama-Tibasosa / RN55,888.0,2.0,80.0,1.0,...,2021-11-30 20:01:26,2021-11-30,Nov,30,20:01:26,1,Tue,0.888,39.960000,0.008019
4,5,Duitama_Sogamoso,28847,Duitama-Tibasosa / RN55-02 >(N),120,d,1665.0,0.0,121.0,0.0,...,2021-11-30 20:01:26,2021-11-30,Nov,30,20:01:26,1,Tue,1.665,49.950000,0.015403
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61534,61535,Sogamoso_Duitama,28848,Duitama-Paipa / RN55-02 >(S),891,Calle 11,14244.0,0.0,891.0,0.0,...,2022-May-06 15:06:52,2022-May-06,May,6,15:06:52,6,Fri,14.244,57.551515,0.128888
61535,61536,Sogamoso_Duitama,28848,Duitama-Paipa / RN55-02 >(S),121,d,1000.0,4.0,204.0,1.0,...,2022-May-06 15:06:52,2022-May-06,May,6,15:06:52,6,Fri,1.000,29.752066,0.009032
61536,61537,Sogamoso_Duitama,28848,d,21,Tibasosa-Duitama / RN62-10 >Occidente,258.0,0.0,21.0,2.0,...,2022-May-06 15:06:52,2022-May-06,May,6,15:06:52,6,Fri,0.258,44.228571,0.002332
61537,61538,Sogamoso_Duitama,28848,Carrera 35,129,Av. de las Américas,1267.0,0.0,132.0,4.0,...,2022-May-06 15:06:52,2022-May-06,May,6,15:06:52,6,Fri,1.267,35.358140,0.011436
